# **Master in Data Science and Computer Engineering**

## Lab: Data-Centric vs Model-Centric approaches

This lab gives an introduction to data-centric vs model-centric approaches to machine learning problems, showing how data-centric approaches can outperform purely model-centric approaches.

In this lab, we'll build a classifier for product reviews (restricted to the magazine category), like:

> *Excellent! I look forward to every issue. I had no idea just how much I didn't know.  The letters from the subscribers are educational, too.*

Label: ⭐️⭐️⭐️⭐️⭐️ (good)

> *My son waited and waited, it took the 6 weeks to get delivered that they said it would but when it got here he was so dissapointed, it only took him a few minutes to read it.*

Label: ⭐️ (bad)

We'll work with a dataset that has some issues, and we'll see how we can squeeze only so much performance out of the model by being clever about model choice, searching for better hyperparameters, etc.

Then, we'll take a look at the data (as any good data scientist should), develop an understanding of the issues, and use simple approaches to improve the data. As such, we'll see how improving the data can improve results.

This lab was partially adapted from ["Introduction Data Centric AI"](https://dcai.csail.mit.edu/) course of MIT University.

## Installing software

For this lab, you'll need to install [scikit-learn](https://scikit-learn.org/) and [pandas](https://pandas.pydata.org/). These are included by default in Google Colab.

However, if you are running this Notebook via Jupyter offline and don't have them installed already in your environment, you can install them by removing the comment and running the following cell:

In [3]:
#!pip install scikit-learn pandas

# Loading the data

First, let's load the train/test sets and take a look at the data.

In [4]:
import pandas as pd

In [5]:
train = pd.read_csv('https://raw.githubusercontent.com/dcai-course/dcai-lab/refs/heads/master/data_centric_model_centric/reviews_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/dcai-course/dcai-lab/refs/heads/master/data_centric_model_centric/reviews_test.csv')

test.sample(5)

,review,label
404,When my grandson chose this magazine at the ne...,good
665,The new Better Homes App on the Kindle is a pa...,bad
511,Does not work on Kindle for WP.,bad
119,"Husband loves this magazine, he looks forward ...",good
610,i ordered men's health for someone incarcerat...,bad


# Training a baseline model

There are many approaches for training a sequence classification model for text data. In this lab, we're giving you code that mirrors what you find if you look up [how to train a text classifier](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html), where we'll train an SVM on [tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) features (numeric representations of each text field based on word occurrences).

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

In [7]:
sgd_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

In [8]:
_ = sgd_clf.fit(train['review'], train['label'])

## Evaluating model accuracy

In [9]:
from sklearn import metrics

In [10]:
def evaluate(clf):
    pred = clf.predict(test['review'])
    acc = metrics.accuracy_score(test['label'], pred)
    print(f'Accuracy: {100*acc:.1f}%')

In [11]:
evaluate(sgd_clf)

Accuracy: 76.5%


## Trying another model

76% accuracy is not great for this binary classification problem. Can you do better with a different model, or by tuning hyperparameters for the SVM trained with SGD?

# Exercise 1

Can you train a more accurate model on the dataset (without changing the dataset)? You might find this [scikit-learn classifier comparison](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html) handy, as well as the [documentation for supervised learning in scikit-learn](https://scikit-learn.org/stable/supervised_learning.html).

One idea for a model you could try is a [naive Bayes classifier](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html).

You could also try experimenting with different values of the model hyperparameters, perhaps tuning them via a [grid search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

Or you can even try training multiple different models and [ensembling their predictions](https://scikit-learn.org/stable/modules/ensemble.html#voting-classifier), a strategy often used to win prediction competitions like Kaggle.

**Advanced:** If you want to be more ambitious, you could try an even fancier model, like training a Transformer neural network. If you go with that, you'll want to fine-tune a pre-trained model. This [guide from HuggingFace](https://huggingface.co/docs/transformers/training) may be helpful.

## Taking a closer look at the training data

Let's actually take a look at some of the training data:

In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

mnb_pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('mnb', MultinomialNB()),
])

best_gnb_model = mnb_pipeline.fit(train['review'], train['label'])
evaluate(best_gnb_model)

Accuracy: 85.3%


In [13]:
train.head()

,review,label
0,Based on all the negative comments about Taste...,good
1,I still have not received this. Obviously I c...,bad
2,</tr>The magazine is not worth the cost of sub...,good
3,This magazine is basically ads. Kindve worthle...,bad
4,"The only thing I've recieved, so far, is the b...",bad


Zooming in on one particular data point:

In [14]:
print(train.iloc[0].to_dict())

{'review': "Based on all the negative comments about Taste of Home, I will not subscribeto the magazine. In the past it was a great read.\nSorry it, too, has gone the 'way of the wind'.<br>o-p28pass4 </br>", 'label': 'good'}


This data point is labeled "good", but it's clearly a negative review. Also, it looks like there's some funny HTML stuff at the end.

# Exercise 2

Take a look at some more examples in the dataset. Do you notice any patterns with bad data points?

In [15]:
# Check 3-4 samples of the training data (both 'good' and 'bad' labels)
for i in range(4):
    print(train[train["label"] == "bad"].iloc[i].to_dict())
    print(train[train["label"] == "good"].iloc[i].to_dict())
    print()


{'review': "I still have not received this.  Obviously I can't review something I haven't seen.  Where is my order???????", 'label': 'bad'}
{'review': "Based on all the negative comments about Taste of Home, I will not subscribeto the magazine. In the past it was a great read.\nSorry it, too, has gone the 'way of the wind'.<br>o-p28pass4 </br>", 'label': 'good'}

{'review': 'This magazine is basically ads. Kindve worthless to me really.', 'label': 'bad'}
{'review': '</tr>The magazine is not worth the cost of subscription.</tr>', 'label': 'good'}

{'review': "The only thing I've recieved, so far, is the bill.\nI know this is [ or was when I last subscribed ] a ten month magazine - but so far not even the usual promotional papers.......", 'label': 'bad'}
{'review': 'The magazines are great, but I never received the golf shoe bag that was supposed to accompany my subscription...<body>', 'label': 'good'}

{'review': 'Did not. Open.', 'label': 'bad'}
{'review': 'This is one magazine I reall

## Issues in the data

It looks like there's some funny HTML tags in our dataset, and those datapoints have nonsense labels. Maybe this dataset was collected by scraping the internet, and the HTML wasn't quite parsed correctly in all cases.

# Exercise 3

To address this, a simple approach we might try is to throw out the bad data points, and train our model on only the "clean" data.

Come up with a simple heuristic to identify data points containing HTML, and filter out the bad data points to create a cleaned training set.

In [16]:
# Complete this function with a "creative" yet simple heuristic
def is_bad_data(review: str) -> bool:
    if review.find("<") != -1:
        return True
    else:
      return False

## Creating the cleaned training set

In [17]:
train_clean = train[~train['review'].map(is_bad_data)]

## Evaluating a model trained on the clean training set

In [18]:
from sklearn import clone

In [19]:
sgd_clf_clean = clone(sgd_clf)

In [20]:
_ = sgd_clf_clean.fit(train_clean['review'], train_clean['label'])

This model should do significantly better:

In [21]:
evaluate(sgd_clf_clean)

Accuracy: 97.1%


# Advanced: Training a Transformer model with HuggingFace

Here we demonstrate how to fine-tune a pretrained Transformer network (variant of BERT) on the original training dataset. You should be able to run the same code on your cleaned up version of the training data.

If you experience installation issues with the following dependencies, consider using a [Colab notebook](https://colab.research.google.com/) instead of your own laptop.

In [22]:
!pip install torch transformers datasets

ERROR: Operation cancelled by user
^C


In [ ]:
import numpy as np
import transformers
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import datasets
from datasets import Dataset, DatasetDict, ClassLabel

We first reformat the data to be suitable with the HuggingFace Dataset class.

In [ ]:
label_map = {"bad": 0, "good": 1}
dataset_train = Dataset.from_dict({"label": train["label"].map(label_map), "text": train["review"].values})
dataset_test = Dataset.from_dict({"label": test["label"].map(label_map), "text": test["review"].values})

Let's set some configurations for our model. You can play with these values to try and improve accuracy.

In [ ]:
model_name = "distilbert-base-uncased"  # which pretrained neural network weights to load for fine-tuning on our data
# other options you could try: "bert-base-uncased", "bert-base-cased", "google/electra-small-discriminator"

max_training_steps = 10  # how many iterations our network will be trained for
# Here set to a tiny value to ensure quick runtimes, set to higher values if you have a GPU to run this code on.

model_folder = "test_trainer"  # file where model will be saved after training

Now we can train our Transformer model with the configuration selected above.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_tokenized_dataset = dataset_train.map(tokenize_function, batched=True)
train_tokenized_dataset = train_tokenized_dataset.cast_column("label", ClassLabel(names = ["0", "1"]))

test_tokenized_dataset = dataset_test.map(tokenize_function, batched=True)
test_tokenized_dataset = test_tokenized_dataset.cast_column("label", ClassLabel(names = ["0", "1"]))

training_args = TrainingArguments(max_steps=max_training_steps, output_dir=model_folder)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Casting the dataset:   0%|          | 0/6666 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()  # may take a while to train (try to run on a GPU if you can access one)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:


Abort: 

Finally we evaluate the Transformer network's accuracy on our test data.

In [ ]:
pred_probs = trainer.predict(test_tokenized_dataset).predictions
pred_classes = np.argmax(pred_probs, axis=1)
print(f"Error rate of predictions: {np.mean(pred_classes != test_tokenized_dataset['label'])}")